# AutoML

In [77]:
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_log_error

import functools

from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularDataset, TabularPredictor

## 读入数据

In [15]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

abalone_cols = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1', 'Whole weight.2', 'Shell weight', 'Rings']
ext_df = pd.read_csv('../input/abalone.data', names=abalone_cols)

# 之前实验已经证明加入原始数据提升明显，因此我们后续实验都使用合并后的训练数据集
train_df = pd.concat([train_df, ext_df], axis=0, ignore_index=True)

train_df.shape[0], test_df.shape[0]

(94792, 60411)

In [16]:
train_df.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0.0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1.0,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2.0,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3.0,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4.0,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


对`Sex`进行编码，以便于输入模型

In [19]:
Sex2code = {'I': 0, 'M': 1, 'F': 2}
train_df['Sex'] = train_df['Sex'].map(Sex2code)
test_df['Sex'] = test_df['Sex'].map(Sex2code)

由于最终的评估指标是`RMSLE`，并且之前的实验已经证明对预测目标(Rings)取对数有效果提升，我们先做掉对数变换

In [20]:
train_df['Rings'] = np.log1p(train_df['Rings'])

In [21]:
train_df.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0.0,2,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,2.484907
1,1.0,2,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,2.484907
2,2.0,0,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,1.945910
3,3.0,1,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,2.397895
4,4.0,0,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,2.302585


In [22]:
origin_cat_feats = ['Sex']
origin_feats = [c for c in train_df.columns if c not in {'id', 'Rings'}]
label = 'Rings'

In [23]:
origin_feats

['Sex',
 'Length',
 'Diameter',
 'Height',
 'Whole weight',
 'Whole weight.1',
 'Whole weight.2',
 'Shell weight']

## LightGBM

In [24]:
def lgb_cv(params, train_df, test_df, train_cols, target_col, nfold=5, num_boost_round=10000):
    folds = KFold(n_splits=nfold, shuffle=True, random_state=42)
    target = train_df[target_col]

    oof = np.zeros(train_df.shape[0])
    pred = np.zeros(test_df.shape[0])
    
    for i, (trn_idx, val_idx) in enumerate(folds.split(train_df.index)):
        print(f'fold={i}', '- ' * 20)
        trn_data = lgb.Dataset(train_df.loc[trn_idx, train_cols], label=target.loc[trn_idx], categorical_feature=['Sex'])
        val_data = lgb.Dataset(train_df.loc[val_idx, train_cols], label=target.loc[val_idx], categorical_feature=['Sex'])
    
        model = lgb.train(params, trn_data, num_boost_round, valid_sets=val_data, callbacks=[lgb.log_evaluation(200)])
    
        oof[val_idx] = model.predict(train_df.loc[val_idx, train_cols], num_iteration=model.best_iteration)
        pred += model.predict(test_df[train_cols], num_iteration=model.best_iteration) / nfold

    oof = np.expm1(oof)
    pred = np.expm1(pred)

    cv = mean_squared_log_error(np.expm1(target), oof, squared=False)
    return cv, oof, pred

In [25]:
lgb_params = {
    'num_threads': 8,
    'objective': 'regression',
    'learning_rate': 0.01,
    'num_leaves': 153,
    'min_data_in_leaf': 79,
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'reg_lambda': 2,
    'metric': 'l2',
    'early_stopping_rounds': 600,
}

In [26]:
cv, oof, pred = lgb_cv(lgb_params, train_df, test_df, origin_feats, label)

fold=0 - - - - - - - - - - - - - - - - - - - - 
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003609 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1333
[LightGBM] [Info] Number of data points in the train set: 75833, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 2.330406
Training until validation scores don't improve for 600 rounds
[200]	valid_0's l2: 0.0243312
[400]	valid_0's l2: 0.0226102
[600]	valid_0's l2: 0.0224015
[800]	valid_0's l2: 0.0223152
[1000]	valid_0's l2: 0.0222839
[1200]	valid_0's l2: 0.0222701
[1400]	valid_0's l2: 0.022268
[1600]	valid_0's l2: 0.022284
[1800]	valid_0's l2: 0.022302
Early stopping, best iteration is:
[1392]	valid_0's l2: 0.0222673
fold=1 - - - - - - - - - - - - - - - - - - - - 
[Li

In [27]:
cv

0.14869038684622946

## 自动特征工程：OpenFE

原理见论文：https://arxiv.org/pdf/2211.12507

### hack

修改源码，增加我们想要的特征变换(分位数)

In [28]:
import openfe
openfe.__file__

'/home/admin/jiazhuang.jz/miniconda3/envs/py38/lib/python3.8/site-packages/openfe/__init__.py'

修改 `/home/admin/jiazhuang.jz/miniconda3/envs/py38/lib/python3.8/site-packages/openfe/FeatureGenerator.py`文件

第6行修改为：
```python
num_operators = ["abs", "log", "sqrt", "square", "sigmoid", "round", "residual", "<p0.2", "<p0.4", "<p0.6", "<p0.8",]
```

在第63行后增加：
```
            elif self.name == '<p0.2':
                new_data = (d < d.quantile(0.2).max()).astype(int)
            elif self.name == '<p0.4':
                new_data = (d < d.quantile(0.4).max()).astype(int)
            elif self.name == '<p0.6':
                new_data = (d < d.quantile(0.6).max()).astype(int)
            elif self.name == '<p0.8':
                new_data = (d < d.quantile(0.8).max()).astype(int)
```

重启 notebook kernel 生效！

### 生成新的特征

In [45]:
from openfe import OpenFE, transform, get_candidate_features, tree_to_formula

In [30]:
origin_num_feats = [c for c in origin_feats if c not in origin_cat_feats]
origin_num_feats

['Length',
 'Diameter',
 'Height',
 'Whole weight',
 'Whole weight.1',
 'Whole weight.2',
 'Shell weight']

In [31]:
train_x, train_y = train_df[origin_feats], train_df[[label]]
test_x = test_df[origin_feats]

In [35]:
candidate_features = get_candidate_features(numerical_features=origin_num_feats, categorical_features=origin_cat_feats)

In [36]:
len(candidate_features)

253

In [37]:
for f in candidate_features:
    print(tree_to_formula(f), '<-', f.name)

freq(Length) <- freq
freq(Diameter) <- freq
freq(Height) <- freq
freq(Whole weight) <- freq
freq(Whole weight.1) <- freq
freq(Whole weight.2) <- freq
freq(Shell weight) <- freq
freq(Sex) <- freq
abs(Length) <- abs
abs(Diameter) <- abs
abs(Height) <- abs
abs(Whole weight) <- abs
abs(Whole weight.1) <- abs
abs(Whole weight.2) <- abs
abs(Shell weight) <- abs
log(Length) <- log
log(Diameter) <- log
log(Height) <- log
log(Whole weight) <- log
log(Whole weight.1) <- log
log(Whole weight.2) <- log
log(Shell weight) <- log
sqrt(Length) <- sqrt
sqrt(Diameter) <- sqrt
sqrt(Height) <- sqrt
sqrt(Whole weight) <- sqrt
sqrt(Whole weight.1) <- sqrt
sqrt(Whole weight.2) <- sqrt
sqrt(Shell weight) <- sqrt
square(Length) <- square
square(Diameter) <- square
square(Height) <- square
square(Whole weight) <- square
square(Whole weight.1) <- square
square(Whole weight.2) <- square
square(Shell weight) <- square
sigmoid(Length) <- sigmoid
sigmoid(Diameter) <- sigmoid
sigmoid(Height) <- sigmoid
sigmoid(Whole 

In [38]:
set(f.name for f in candidate_features)

{'*',
 '+',
 '-',
 '/',
 '<p0.2',
 '<p0.4',
 '<p0.6',
 '<p0.8',
 'GroupByThenMax',
 'GroupByThenMean',
 'GroupByThenMedian',
 'GroupByThenMin',
 'GroupByThenRank',
 'GroupByThenStd',
 'abs',
 'freq',
 'log',
 'max',
 'min',
 'residual',
 'round',
 'sigmoid',
 'sqrt',
 'square'}

In [39]:
# Restrict Search Space of Candidate Features
candidate_features = [
    f
    for f in candidate_features
    if f.name
    in {
        # "abs" -> dataset specific, not useful in most cases
        # "log" -> can be done by scalers, no need for GBDTs
        # "sqrt", -> see above (s.a.)
        # "square" , -> s.a.
        # "sigmoid" , -> s.a.
        "freq",
        "round",
        "residual",
        # "max", -> IMO, trivial to model for first-order features 
        # "min", -> s.a.
        "+",
        "-",
        "/",
        "*",
        "GroupByThenMin",  # -> &nbsp;The essential benefit of GroupBy is captured with any of these, so I filtered this to reduce the search space.
        "GroupByThenMax",  # -> s.a.
        # "GroupByThenMean", -> s.a.
        "GroupByThenMedian",
        "GroupByThenStd",
        "GroupByThenRank",
        "GroupByThenFreq",
        "GroupByThenNUnique",
        "Combine",
        # New Generators 
        #   - Hacked into OpenFE by adding `new_data = int(d < d.quantile(X).max())` to the generator options. 
        "<p0.2",  # X = 0.2
        "<p0.4",   
        "<p0.6",
        "<p0.8",
    }
]

In [40]:
len(candidate_features)

169

In [41]:
ofe = OpenFE()

In [42]:
%%capture --no-stderr

features = ofe.fit(
    data=train_x,
    label=train_y,
    task='regression',
    candidate_features_list=candidate_features,
    metric='rmse',
    n_data_blocks=2,
    min_candidate_features=30000,
    n_jobs=8,
    verbose=True
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:04<00:00,  6.76it/s]


In [43]:
len(features)

138

In [46]:
train_x, test_x = transform(train_x, test_x, features, n_jobs=8)

In [47]:
train_x.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,autoFE_f_0,autoFE_f_1,...,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137
openfe_index,,,,,,,,,,,,,,,,,,,,,
0,2,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,2.291667,0.036000,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.815,0.0,0.0
1,2,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,1.968750,0.046400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.815,1.0,0.0
2,0,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,32.000000,0.000125,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.770,1.0,1.0
3,1,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,2.380000,0.037500,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.815,0.0,0.0
4,0,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,2.810127,0.025675,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.770,1.0,0.0


In [48]:
test_x.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,autoFE_f_0,autoFE_f_1,...,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137
0,1,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,2.146423,0.046578,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.815,0.0,0.0
1,1,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,2.109091,0.044000,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.815,0.0,0.0
2,1,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,2.328482,0.033670,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.815,1.0,0.0
3,1,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,2.425532,0.034075,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.815,1.0,0.0
4,0,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,3.952381,0.011550,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.770,1.0,1.0


In [49]:
fe_train_df = pd.concat([train_x, train_y], axis=1)

In [53]:
fe_test_df = test_x

In [50]:
fe_train_df.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,autoFE_f_0,autoFE_f_1,...,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137,Rings
openfe_index,,,,,,,,,,,,,,,,,,,,,
0,2,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,2.291667,0.036000,...,0.0,0.0,0.0,1.0,1.0,1.0,0.815,0.0,0.0,2.484907
1,2,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,1.968750,0.046400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.815,1.0,0.0,2.484907
2,0,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,32.000000,0.000125,...,1.0,1.0,1.0,1.0,1.0,1.0,0.770,1.0,1.0,1.945910
3,1,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,2.380000,0.037500,...,0.0,0.0,0.0,1.0,1.0,0.0,0.815,0.0,0.0,2.397895
4,0,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,2.810127,0.025675,...,0.0,0.0,0.0,1.0,1.0,1.0,0.770,1.0,0.0,2.302585


In [51]:
fe_feats = [c for c in fe_train_df.columns if c not in {'Rings'}]
len(fe_feats)

146

In [54]:
cv, oof, pred = lgb_cv(lgb_params, fe_train_df, fe_test_df, fe_feats, label)

fold=0 - - - - - - - - - - - - - - - - - - - - 
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26393
[LightGBM] [Info] Number of data points in the train set: 75833, number of used features: 146
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 2.330406
Training until validation scores don't improve for 600 rounds
[200]	valid_0's l2: 0.0233625
[400]	valid_0's l2: 0.0219381
[600]	valid_0's l2: 0.0218334
[800]	valid_0's l2: 0.0218092
[1000]	valid_0's l2: 0.0218141
[1200]	valid_0's l2: 0.0218379
[1400]	valid_0's l2: 0.0218637
Early stopping, best iteration is:
[918]	valid_0's l2: 0.0218016
fold=1 - - - - - - - - - - - - - - - - - - - - 
[LightGBM] [Warning] Found whitespace in feature_names, repl

In [55]:
cv

0.14707343774411905

## 特征选择：AutoGluon

论文：https://arxiv.org/pdf/2003.06505

### hack

AutoGluon在计算 LightGBM 模型 metrics 时有bug，需要手动修复

In [59]:
import autogluon.tabular
print(autogluon.tabular.__file__)

/home/admin/jiazhuang.jz/miniconda3/envs/py38/lib/python3.8/site-packages/autogluon/tabular/__init__.py


将文件`/home/admin/jiazhuang.jz/miniconda3/envs/py38/lib/python3.8/site-packages/autogluon/tabular/models/lgb/lgb_utils.py`的41行替换为：
```python
    if problem_type in [REGRESSION, QUANTILE]:
        # TODO: Might not work for custom quantile metrics
        def function_template(y_hat, data):
            y_true = data.get_label()
            return metric.name, metric(y_true, y_hat), is_higher_better  

    elif needs_pred_proba:
```
重启 notebook kernel 生效！

### 特征选择

AutoGluon的特征选择/修剪(feature_prune)功能在实验阶段，没有直接的API，这里的使用方法有些hack

In [73]:
hyperparameters = {'GBM':{}}
feature_prune_kwargs = {'feature_prune_time_limit': 600, 'n_train_subsample': None, 'n_fi_subsample': 20000, 'prune_ratio': 0.05, 'stopping_round': 20, 'min_improvement': 1e-6, 'force_prune': True}

predictor = TabularPredictor(
    label=label,
    problem_type='regression',
    eval_metric='rmse',
).fit(
    fe_train_df[fe_feats + ['Rings']],
    hyperparameters=hyperparameters,
    feature_prune_kwargs=feature_prune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240510_164345"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240510_164345"
=================== System

[1000]	valid_set's rmse: 0.146474


	Round 4 of feature pruning model fit (7.54s):
		Validation score of the current model fit on 134 features (-0.1464) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.
	Round 5 of feature pruning model fit (5.92s):
		Validation score of the current model fit on 134 features (-0.1463) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.
	Round 6 of feature pruning model fit (6.07s):
		Validation score of the current model fit on 134 features (-0.1465) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.


[1000]	valid_set's rmse: 0.146311


	Round 7 of feature pruning model fit (8.18s):
		Validation score of the current model fit on 134 features (-0.1462) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.
	Round 8 of feature pruning model fit (5.29s):
		Validation score of the current model fit on 134 features (-0.1465) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.
	Round 9 of feature pruning model fit (4.5s):
		Validation score of the current model fit on 134 features (-0.1467) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.
	Round 10 of feature pruning model fit (5.62s):
		Validation score of the current model fit on 134 features (-0.1463) is not better than validation score of the best model fit on 141 features (-0.1462). Retrying.
	Round 11 of feature pruning model fit (5.31s):
		Validation score of the current model fit on 134 features (-0.1464) is not better than validation scor

...

	Round 27 of feature pruning model fit (7.15s):
		Validation score of the current model fit on 111 features (-0.1463) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.
	Round 28 of feature pruning model fit (4.41s):
		Validation score of the current model fit on 111 features (-0.1465) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.
	Round 29 of feature pruning model fit (5.43s):
		Validation score of the current model fit on 111 features (-0.1465) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.


[1000]	valid_set's rmse: 0.146491


	Round 30 of feature pruning model fit (6.45s):
		Validation score of the current model fit on 111 features (-0.1463) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.
	Round 31 of feature pruning model fit (5.07s):
		Validation score of the current model fit on 111 features (-0.1464) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.
	Round 32 of feature pruning model fit (5.97s):
		Validation score of the current model fit on 111 features (-0.1464) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.


[1000]	valid_set's rmse: 0.146528
[1000]	valid_set's rmse: 0.146346


	Round 33 of feature pruning model fit (7.37s):
		Validation score of the current model fit on 111 features (-0.1463) is not better than validation score of the best model fit on 116 features (-0.146). Retrying.
	Round 34 of feature pruning model fit (5.83s):
		Validation score of the current model fit on 111 features (-0.1459) is better than validation score of the best model fit on 116 features (-0.146). Updating model.
	Round 35 of feature pruning model fit (2.98s):
		Validation score of the current model fit on 106 features (-0.147) is not better than validation score of the best model fit on 111 features (-0.1459). Retrying.
	Time is up while computing feature importance or there are no more features to prune. Ending...
	Successfully ended prune loop after 36 feature pruning rounds (262.45s).
	Feature count before/after feature pruning: 141 -> 111.
Fitting model: LightGBM_Prune ...
	-0.1462	 = Validation score   (-root_mean_squared_error)
	4.51s	 = Training   runtime
	0.01s	 = Val

In [76]:
ls AutogluonModels/ag-20240510_164345/models/LightGBM_Prune

model.pkl


In [78]:
with open('AutogluonModels/ag-20240510_164345/models/LightGBM_Prune/model.pkl', 'rb') as f:
    prune_model = pickle.load(f)

In [79]:
selected_feats = prune_model.features

In [81]:
len(selected_feats)

111

In [82]:
cv, oof, pred = lgb_cv(lgb_params, fe_train_df, fe_test_df, selected_feats, label)

fold=0 - - - - - - - - - - - - - - - - - - - - 
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020750 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22051
[LightGBM] [Info] Number of data points in the train set: 75833, number of used features: 111
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 2.330406
Training until validation scores don't improve for 600 rounds
[200]	valid_0's l2: 0.0233533
[400]	valid_0's l2: 0.0219567
[600]	valid_0's l2: 0.021831
[800]	valid_0's l2: 0.0218064
[1000]	valid_0's l2: 0.0218137
[1200]	valid_0's l2: 0.0218323
[1400]	valid_0's l2: 0.0218553
Early stopping, best iteration is:
[812]	valid_0's l2: 0.0218034
fold=1 - - - - - - - - - - - - - - - - - - - - 
[LightGBM] [Warning] Found whitespace in feature_names, repla

In [83]:
cv

0.1470460411601835

## 模型选择&融合：AutoGluon

In [84]:
clean_train_df = fe_train_df[selected_feats + [label]]
clean_test_df = fe_test_df[selected_feats]

In [ ]:
predictor = TabularPredictor(
    label=label,
    problem_type='regression',
    eval_metric='rmse',
).fit(clean_train_df, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240510_165545"
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240510_165545/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


In [92]:
result_df = predictor.leaderboard(extra_info=True)
result_df

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L3,-0.146603,root_mean_squared_error,82.697784,2059.812785,0.001871,0.371878,3,True,35,...,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetFastAI_BAG_L2, LightGBM_r96_BAG_L1, ExtraTreesMSE_BAG_L2, CatBoost_r9_BAG_L1, LightGBM_BAG_L2, LightGBM_r131_BAG_L1, CatBoost_BAG_L2, RandomForestMSE_BAG_L2]",None,"{'ensemble_size': 25, 'subsample_size': 1000000}",{'ensemble_size': 19},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetTorch_r79_BAG_L1, CatBoost_r177_BAG_L1, LightGBM_BAG_L1, NeuralNetFastAI_r191_BAG_L1, LightGBMLarge_BAG_L1, NeuralNetTorch_BAG_L1, RandomForestMSE_BAG_L1, XGBoost_r33_BAG_L1, NeuralNetFastAI_BAG_L2, ExtraTreesMSE_BAG_L1, CatBoost_r9_BAG_L1, LightGBM_BAG_L2, RandomForestMSE_BAG_L2, LightGBM_r131_BAG_L1, CatBoost_BAG_L1, ExtraTreesMSE_BAG_L2, KNeighborsUnif_BAG_L1, LightGBMXT_BAG_L1, CatBoost_BAG_L2, XGBoost_BAG_L1, KNeighborsDist_BAG_L1, LightGBM_r96_BAG_L1, NeuralNetTorch_r22_BAG_L1, NeuralNetFastAI_BAG_L1]",[]
1,WeightedEnsemble_L2,-0.146670,root_mean_squared_error,37.688936,589.743959,0.001885,0.291966,2,True,20,...,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[LightGBM_BAG_L1, CatBoost_BAG_L1, LightGBM_r96_BAG_L1, ExtraTreesMSE_BAG_L1, CatBoost_r9_BAG_L1, LightGBM_r131_BAG_L1, NeuralNetFastAI_BAG_L1, LightGBMLarge_BAG_L1, RandomForestMSE_BAG_L1]",None,"{'ensemble_size': 25, 'subsample_size': 1000000}",{'ensemble_size': 23},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[LightGBM_BAG_L1, CatBoost_BAG_L1, LightGBM_r96_BAG_L1, ExtraTreesMSE_BAG_L1, CatBoost_r9_BAG_L1, LightGBM_r131_BAG_L1, NeuralNetFastAI_BAG_L1, LightGBMLarge_BAG_L1, RandomForestMSE_BAG_L1]",[]
2,NeuralNetFastAI_BAG_L2,-0.146696,root_mean_squared_error,61.073026,1850.082259,1.679635,132.392699,2,True,26,...,"{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_bat

In [100]:
cols = ['model', 'score_val']
result_df[cols]

,model,score_val
0,WeightedEnsemble_L3,-0.146603
1,WeightedEnsemble_L2,-0.146670
2,NeuralNetFastAI_BAG_L2,-0.146696
3,CatBoost_BAG_L2,-0.146915
4,LightGBM_BAG_L2,-0.146943
5,CatBoost_r177_BAG_L2,-0.146955
6,LightGBMXT_BAG_L2,-0.146956
7,LightGBM_r131_BAG_L2,-0.147008
8,LightGBM_r131_BAG_L1,-0.147021
9,CatBoost_r9_BAG_L1,-0.147031


In [ ]:
pred = predictor.predict(test_data)

In [101]:
sub_df = pd.read_csv('../input/sample_submission.csv')

sub_df['Rings'] = np.expm1(pred)

cols = ['id', 'Rings']
sub_df[cols].to_csv('autogluon_score_1466.csv', index=False, header=True)

In [102]:
!head autogluon_score_1466.csv

id,Rings
90615,9.616958
90616,9.836972
90617,9.399967
90618,10.396208
90619,7.714531
90620,9.294041
90621,11.499154
90622,6.1690116
90623,7.9552164
